<a href="https://colab.research.google.com/github/Sanika2029/Sanika/blob/main/gst_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [38]:
#importing dataset and dropping ID , col 9
xtrain = pd.read_csv('/content/X_Train_Data_Input.csv')
ytrain = pd.read_csv('/content/Y_Train_Data_Target.csv')
xtest = pd.read_csv('/content/X_Test_Data_Input.csv')
ytest = pd.read_csv('/content/Y_Test_Data_Target.csv')
xtrain = xtrain.drop('ID', axis=1)
xtest = xtest.drop('ID', axis=1)
ytrain = ytrain.drop('ID',axis=1)
ytest = ytest.drop('ID', axis=1)
xtrain = xtrain.drop('Column9', axis=1)
xtest = xtest.drop('Column9',axis=1)

In [39]:
ytest.shape

(261712, 1)

In [40]:
xtest.shape

(261712, 21)

In [41]:
xtrain.shape

(785133, 21)

In [42]:
ytrain.shape

(785133, 1)

In [43]:
 #missing values
MVC = ['Column0','Column3','Column4','Column5','Column6','Column8','Column14','Column15']

for column in MVC:
    xtrain[column] = xtrain[column].fillna(method='ffill')
for column in MVC:
    xtest[column] = xtest[column].fillna(method='ffill')

In [44]:
#y train n test datafram to array
ytrain = ytrain.values.ravel()
ytest = ytest.values.ravel()

In [49]:
xtest.shape

(261712, 21)

In [48]:
xtrain.shape

(785133, 21)

In [45]:
ytrain.shape

(785133,)

In [46]:
ytest.shape

(261712,)

In [50]:
xtrain.isnull().sum()

,0
Column0,0
Column1,0
Column2,0
Column3,0
Column4,0
Column5,0
Column6,0
Column7,0
Column8,0
Column10,0


In [51]:
#outliers removing and making back to df
def remove_outliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 + IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
        return df
xtrain_cl = remove_outliers(xtrain, xtrain.columns)
xtrain_cl = pd.DataFrame(xtrain_cl, columns=xtrain.columns)
ytrain_cl = ytrain[xtrain_cl.index]
ytrain_cl = ytrain_cl.reshape(-1,1)
ytrain_cl = pd.DataFrame(ytrain_cl)

In [54]:
xtrain.shape

(785133, 21)

In [53]:
xtrain_cl.shape

(685587, 21)

In [55]:
ytrain.shape

(785133,)

In [52]:
ytrain_cl.shape

(685587, 1)

In [56]:
from sklearn.decomposition import PCA

# Create a PCA object with 5 components (you can adjust this number)
pca = PCA(n_components=10)

# Fit and transform the data
X_pca = pca.fit_transform(xtrain_cl)

# Convert the transformed data back to a DataFrame
X_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])

print(X_pca.shape)

(685587, 10)


In [57]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Assuming 'X_pca' is your DataFrame with categorical features

# Separate numerical and categorical features
numerical_features = X_pca.select_dtypes(include=['number']).columns
categorical_features = X_pca.select_dtypes(include=['object']).columns

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the numerical features
X_pca[numerical_features] = scaler.fit_transform(X_pca[numerical_features])

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform the categorical features
encoded_features = encoder.fit_transform(X_pca[categorical_features])

# Create a new DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features)

# Concatenate the encoded features with the scaled numerical features
X_pca = pd.concat([X_pca[numerical_features], encoded_df], axis=1)

# Now X_pca contains both scaled numerical features and one-hot encoded categorical features

In [58]:
X_pca.shape

(685587, 10)

In [59]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Create a RandomForestClassifier with specified parameters
rf_classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced')

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_classifier, X_pca, ytrain_cl.values.ravel(), cv=5)

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())


Cross-validation scores: [0.96980703 0.96999665 0.97045589 0.9696026  0.9703392 ]
Mean cross-validation score: 0.9700402724626812


In [61]:
rf_classifier.fit(X_pca, ytrain_cl.values.ravel())

RandomForestClassifier(class_weight='balanced')

In [67]:
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'xtest' is your test data
# Apply PCA
X_test_pca = pca.transform(xtest)
X_test_pca = pd.DataFrame(X_test_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])

# Scale numerical features
X_test_pca[numerical_features] = scaler.transform(X_test_pca[numerical_features])

# Encode categorical features
encoded_test_features = encoder.transform(X_test_pca[categorical_features])
encoded_test_df = pd.DataFrame(encoded_test_features)
X_test_pca = pd.concat([X_test_pca[numerical_features], encoded_test_df], axis=1)


# Make predictions on the test data
ypred = rf_classifier.predict(X_test_pca)

# Generate the classification report
print(classification_report(ytest, ypred))

# Generate the confusion matrix
print(confusion_matrix(ytest, ypred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98    237034
           1       0.84      0.88      0.86     24678

    accuracy                           0.97    261712
   macro avg       0.91      0.93      0.92    261712
weighted avg       0.97      0.97      0.97    261712

[[232758   4276]
 [  3030  21648]]
